## IMPORTING THE NECCESSARY LIBRARIES

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from sklearn.model_selection import train_test_split
import shutil
import os

import csv

In [48]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## DATA LOADING AND PREPROCESSING

In [49]:
train_csv = pd.read_csv('Train.csv')
test_csv = pd.read_csv('Test.csv')
meta_csv = pd.read_csv('Meta.csv')

In [12]:
train_csv.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


#### PROPROCESSING THE DATA AND CREATING IMAGE DATA GENERATORS FOR TRAINING

In [13]:
# this function splits the training images into training and validation images
def split_data(data_path, train_data_path, val_data_path, split_size = 0.1):

    folders = os.listdir(data_path)

    for folder in folders:
        full_path = os.path.join(data_path, folder)
        images_path = glob.glob(os.path.join(full_path, '*.png'))

        x_train, x_val = train_test_split(images_path, test_size=split_size)

        # train images
        for x in x_train:
            # basename = os.path.basename(x)
            folder_path = os.path.join(train_data_path, folder)

            if not os.path.isdir(folder_path):
                os.makedirs(folder_path)

            shutil.copy(x, folder_path)

            # validation images
        for x in x_val:
            folder_path = os.path.join(val_data_path, folder)

            if not os.path.isdir(folder_path):
                os.makedirs(folder_path)

            shutil.copy(x, folder_path)


In [14]:
data_path = '/home/elijah/GermanTrafficSignsDetection/Train/'
train_data_path = '/home/elijah/GermanTrafficSignsDetection/training_data/train/'
val_data_path = '/home/elijah/GermanTrafficSignsDetection/training_data/validation/'

In [ ]:
split_data(data_path, train_data_path, val_data_path)

In [15]:
# function to orderly put test images into their respective class labels
def order_test(image_path, csv_path):
    testset = {}

    try:
        with open(csv_path, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')

            for i, row in enumerate(reader):
                if i==0:
                    continue
                image_name = row[-1].replace('Test/', '')
                label = row[-2]
                path_to_folder = os.path.join(image_path, label)

                if not os.path.isdir(path_to_folder):
                    os.makedirs(path_to_folder)
                image_full_path = os.path.join(image_path, image_name)
                shutil.move(image_full_path, path_to_folder)
    except:
        print('[INFO] : Error reading csv file')


In [12]:
order_test(image_path='/home/elijah/GermanTrafficSignsDetection/Test/',
            csv_path='/home/elijah/GermanTrafficSignsDetection/Test.csv')

In [17]:
# the data generators to create labelled images and put them in batches for training the CNN
def create_generators(batch_size, traindata, valdata, testdata):
    preprocessor = ImageDataGenerator(
        rescale=1/255,
    )
    train_generator = preprocessor.flow_from_directory(
        traindata,
        class_mode='categorical',
        target_size=(60,60),
        color_mode='rgb',
        shuffle=True,
        batch_size= batch_size
    )

    validation_generator = preprocessor.flow_from_directory(
        valdata,
        class_mode='categorical',
        target_size=(60,60),
        color_mode='rgb',
        shuffle=False,
        batch_size= batch_size
    )

    test_generator = preprocessor.flow_from_directory(
        testdata,
        class_mode='categorical',
        target_size=(60,60),
        color_mode='rgb',
        shuffle=False,
        batch_size= batch_size
    )

    return train_generator, validation_generator, test_generator




In [37]:
traindata = '/home/elijah/GermanTrafficSignsDetection/training_data/train/'
valdata = '/home/elijah/GermanTrafficSignsDetection/training_data/validation/'
testdata = '/home/elijah/GermanTrafficSignsDetection/Test/'
batch_size = 50

train_generator, validation_generator, test_generator = create_generators(
    batch_size,
    traindata,
    valdata,
    testdata
)
num_classes = train_generator.num_classes

Found 37745 images belonging to 43 classes.
Found 6301 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


## TRAINING AND EVALUATING

In [24]:
from Models import StreetSignDetector


In [38]:
model = StreetSignDetector(num_classes=num_classes)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 60, 60, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 58, 58, 64)        1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 27, 27, 128)       73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 13, 13, 128)       512 

In [39]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',]
)

In [40]:
model_path = './Models'

In [43]:
ckpt_saver = ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=2)

In [44]:
model.fit(
    train_generator,
    epochs=3,
    batch_size=batch_size,
    validation_data=validation_generator,
    callbacks=[ckpt_saver, early_stopping]
)

Epoch 1/3
755/755 [==============================] - 695s 921ms/step - loss: 0.0287 - accuracy: 0.9918 - val_loss: 0.1069 - val_accuracy: 0.9805

Epoch 00001: val_accuracy improved from -inf to 0.98048, saving model to ./Models
INFO:tensorflow:Assets written to: ./Models/assets
Epoch 2/3
755/755 [==============================] - 692s 916ms/step - loss: 0.0239 - accuracy: 0.9935 - val_loss: 0.0137 - val_accuracy: 0.9956

Epoch 00002: val_accuracy improved from 0.98048 to 0.99556, saving model to ./Models
INFO:tensorflow:Assets written to: ./Models/assets
Epoch 3/3
755/755 [==============================] - 713s 945ms/step - loss: 0.0233 - accuracy: 0.9941 - val_loss: 0.1159 - val_accuracy: 0.9678

Epoch 00003: val_accuracy did not improve from 0.99556


In [45]:
model.evaluate(validation_generator)

127/127 [==============================] - 24s 191ms/step - loss: 0.1159 - accuracy: 0.9678


[0.11592686176300049, 0.9677829146385193]

In [46]:
model.evaluate(test_generator)

253/253 [==============================] - 63s 249ms/step - loss: 0.5277 - accuracy: 0.9131


[0.5276893973350525, 0.9130641222000122]